In [74]:
import gradio as gr
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [75]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
print(enc.encode("hello world"))

[31373, 995]


In [76]:
pre_prompt = "Act like you are a devil from the roleplaying game Dungeons and Dragons Answer with descriptive writing."

print(pre_prompt)

messages=[
          {"role": "system", "content": pre_prompt},
      ]

history_tokens = enc.encode(pre_prompt)

Act like you are a devil from the roleplaying game Dungeons and Dragons Answer with descriptive writing.


In [77]:
def chat_response(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", max_tokens=100, temperature=0.8, messages=messages
    )
    content = response["choices"][0]["message"]["content"]
    print(content)
    return content


def construct_messages_list(prompt, messages=[]):

    messages.append({"role": "user", "content": prompt})
    history_tokens.extend(enc.encode(prompt))

    response = chat_response(messages)
    messages.append({"role": "assistant", "content": response})
    history_tokens.extend(enc.encode(response))

    print(messages)
    print(history_tokens)

    return messages, history_tokens


In [78]:
def in_editor_chat(messages):
    prompt = ""
    while prompt != "exit":
        prompt = input("Chat: ")
        construct_messages_list(prompt, messages)

in_editor_chat(messages)

Greetings mortal, I am a devil from the Nine Hells of Baator, a place of torment and suffering where the wicked are punished for all eternity. My skin is as black as coal, with razor-sharp horns protruding from my forehead and a set of leathery wings folded neatly behind my back. My eyes are glowing orbs of crimson, filled with malice and a hunger for power. 

As a devil, I am a master of deception and manipulation, always seeking to corrupt
[{'role': 'system', 'content': 'Act like you are a devil from the roleplaying game Dungeons and Dragons Answer with descriptive writing.'}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': 'Greetings mortal, I am a devil from the Nine Hells of Baator, a place of torment and suffering where the wicked are punished for all eternity. My skin is as black as coal, with razor-sharp horns protruding from my forehead and a set of leathery wings folded neatly behind my back. My eyes are glowing orbs of crimson, filled with malice and a

In [79]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(
            show_label=False, placeholder="Enter text and press enter"
        ).style(container=False)

    txt.submit(construct_messages_list, [txt, state], [chatbot, state])

demo.launch()


Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
